<a href="https://colab.research.google.com/github/2k0v11/ML_Projects/blob/main/text2speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install git+https://github.com/huggingface/transformers.git


In [ ]:
pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.3 MB/s eta 0:00:00


In [ ]:
pip install torchaudio

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")


In [ ]:
inputs = processor(text="Kindness trumps everything. Kind people are magnets for all of the good things in life.", return_tensors="pt")
inputs

{'input_ids': tensor([[ 4, 60, 10,  9, 14,  9,  5, 12, 12,  4,  6, 13, 16, 18, 24, 12,  4,  5,
         27,  5, 13, 22,  6, 11, 10,  9, 21, 26,  4, 60, 10,  9, 14,  4, 24,  5,
          8, 24, 15,  5,  4,  7, 13,  5,  4, 18,  7, 21,  9,  5,  6, 12,  4, 19,
          8, 13,  4,  7, 15, 15,  4,  8, 19,  4,  6, 11,  5,  4, 21,  8,  8, 14,
          4,  6, 11, 10,  9, 21, 12,  4, 10,  9,  4, 15, 10, 19,  5, 26,  2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

import torch
speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)


Generating validation split: 0 examples [00:00, ? examples/s]

Dataset cmu-arctic-xvectors downloaded and prepared to /root/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/a62fea1f9415e240301ea0042ffad2a3aadf4d1caa7f9a8d9512d631723e781f. Subsequent calls will reuse this data.


In [ ]:
spectrogram = model.generate_speech(inputs["input_ids"], speaker_embeddings)
spectrogram

tensor([[-1.8883, -1.9409, -1.9595,  ..., -2.7658, -2.8272, -3.0146],
        [-1.6621, -1.7113, -1.7618,  ..., -2.5724, -2.6272, -2.8526],
        [-1.6953, -1.8064, -1.8255,  ..., -2.7535, -2.7877, -3.0972],
        ...,
        [-2.3707, -2.5615, -2.6486,  ..., -3.6770, -3.7488, -4.0305],
        [-2.4311, -2.6180, -2.7104,  ..., -3.8733, -3.9390, -4.2062],
        [-2.4622, -2.6630, -2.7714,  ..., -3.9395, -3.9982, -4.2469]])

In [ ]:
from transformers import SpeechT5HifiGan
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")


In [ ]:
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)


In [ ]:
import soundfile as sf
sf.write("output.wav", speech.numpy(), samplerate=16000)
